### Guide for Creating Jupyter Notebook Kernal
1. Creating a Virtual Environment
First, you need to create a virtual environment. You can do this using venv (standard library in Python 3.3 and later) or virtualenv (if you are using an older version of Python or prefer it). Here, we'll use venv.

Open your terminal or command prompt and run the following commands:
```python -m venv env```

2. Activating the Virtual Environment
Before installing packages or running Python commands in your virtual environment, you need to activate it:

Windows ```.\env\Scripts\activate```

Mac ```source env/bin/activate```

3. Creating a Jupyter Kernel for the Virtual Environment

Install ipykernel if not already installed
```pip install ipykernel```

Create a Jupyter kernel for the virtual environment
```python -m ipykernel install --user --name=myenv --display-name="trulens kernal"```

Install Libraries

In [ ]:
#! pip install trulens_eval==0.22.2 chromadb==0.4.18 openai==1.3.7 ipython==8.21.0 ipywidgets==8.1.2 langchain-openai == 0.0.6 faiss-cpu==1.7.4 langchainhub==0.1.14

Add API keys
For this quickstart you will need an OpenAI Key.

In [1]:
import os
os.environ["AZURE_OPENAI_API_KEY"] = "<azure-key>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<azure-endpoint>"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_TYPE"] = "azure"

## TruLens Evaluations

Import from TruLens

In [2]:
# Create openai client
from langchain_openai import AzureChatOpenAI

# Imports main tools:
from trulens_eval import Feedback, OpenAI as fOpenAI, Tru
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


Create a simple chat completion function <br> A question answer bot 

In [3]:
def llm_open_ai(prompt):
    llm = AzureChatOpenAI(model_name='gpt-35-turbo',
                            temperature=0)
    return llm.predict(prompt)

Define prompt and call function

In [8]:
prompt_input= "What is photo synthesis?"
prompt_output = llm_open_ai(prompt_input)
prompt_output

/Users/gaurav.shivhare/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Photosynthesis is the process by which green plants, algae, and some bacteria convert sunlight, carbon dioxide, and water into glucose (a type of sugar) and oxygen. It is a vital process for the survival of plants and other organisms that depend on them for food and oxygen. During photosynthesis, chlorophyll (a pigment found in chloroplasts) captures sunlight energy, which is then used to convert carbon dioxide and water into glucose. Oxygen is released as a byproduct of this process. Photosynthesis is responsible for maintaining the oxygen levels in the Earth's atmosphere and is the primary source of energy for most ecosystems."

## Feedback Functions

**Initialize Feedback Functions** <br>
Feed Back Function Guide
https://www.trulens.org/trulens_eval/feedback_function_guide/

In [5]:
from trulens_eval.feedback.provider import AzureOpenAI
import numpy as np
# Initialize AzureOpenAI-based feedback function collection class:
azopenai = AzureOpenAI(
    deployment_name="gpt-35-turbo")

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(azopenai.relevance, name = "Answer Relevance").on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


Want to know more feedback functions? <br>
visit there [git](https://github.com/truera/trulens/tree/main/trulens_eval) <br> trulens_eval/trulens_eval/feedback/v2/feedback.py

Instrument the callable for logging with TruLens

In [9]:
from trulens_eval import TruBasicApp
tru_llm_standalone_recorder = TruBasicApp(llm_open_ai, app_id="Trulens-demo", feedbacks=[f_qa_relevance])

In [10]:
with tru_llm_standalone_recorder as recording:
    tru_llm_standalone_recorder.app(prompt_input)


Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


Scores

In [9]:
tru.get_records_and_feedback(app_ids=[])[0][['input','output','Answer Relevance']]

,input,output,Answer Relevance
0,"""What is photo synthesis?""","""Photosynthesis is the process by which green ...",1.0


Complete Results

In [10]:
tru.get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Answer Relevance_calls,latency,total_tokens,total_cost
0,Training_20Feb,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_3f662b4be16f606cd4a715cedb637508,"""What is photo synthesis?""","""Photosynthesis is the process by which green ...",-,"{""record_id"": ""record_hash_3f662b4be16f606cd4a...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-20T13:28:44.139906"", ""...",2024-02-20T13:28:45.246642,1.0,[{'args': {'prompt': 'What is photo synthesis?...,1,0,0.0


A dashboard?


In [26]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.0.170:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Don't want to use recorder ?

In [12]:
prompt_input = "What is the process of photosynthesis?"
Relevant_Answer = "Photosynthesis is a process used by plants, algae, and certain bacteria to convert light energy into chemical energy stored in glucose or other sugars. This process involves the absorption of carbon dioxide (CO2), water (H2O), and light energy, primarily from the sun. Through a series of reactions known as the light-dependent and light-independent (Calvin cycle) reactions, these inputs are transformed into glucose and oxygen (O2). The overall chemical equation for photosynthesis can be summarized as 6CO2 + 6H2O + light energy → C6H12O6 + 6O2."
Irrelevant_Answer = "Photosynthesis is a method used by animals to digest food. It involves the stomach and intestines breaking down food into nutrients that the body can use for energy, growth, and repair. This process is crucial for the survival of animals and requires a complex digestive system."
print('Good Answer',azopenai.relevance(prompt_input, Relevant_Answer))
print('Bad Answer',azopenai.relevance(prompt_input, Irrelevant_Answer))

Good Answer 1.0
Bad Answer 0.0


## Other Evaluation Metrics
https://www.trulens.org/trulens_eval/api/providers/

In [14]:
from trulens_eval import Feedback
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.feedback.provider import Huggingface
# Initialize provider classes
provider = azopenai = AzureOpenAI(
    deployment_name="gpt-35-turbo")
hugs_provider = Huggingface()

# LLM-based feedback functions
f_coherence = Feedback(
    provider.coherence_with_cot_reasons, name="Coherence"
    ).on_output()

f_input_sentiment = Feedback(
    provider.sentiment_with_cot_reasons, name="Input Sentiment"
    ).on_input()

f_output_sentiment = Feedback(
    provider.sentiment_with_cot_reasons, name="Output Sentiment"
    ).on_output()
        
f_langmatch = Feedback(
    hugs_provider.language_match, name="Language Match"
    ).on_input_output()

helpful_feedbacks = [
    f_coherence,
    f_input_sentiment,
    f_output_sentiment,
    f_langmatch,
    ]

✅ In Coherence, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Input Sentiment, input text will be set to __record__.main_input or `Select.RecordInput` .
✅ In Output Sentiment, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Language Match, input text1 will be set to __record__.main_input or `Select.RecordInput` .
✅ In Language Match, input text2 will be set to __record__.main_output or `Select.RecordOutput` .


In [15]:
helpful_evals = [
    "What types of insurance are commonly used to protect against property damage?",
    "¿Cuál es la diferencia entre un seguro de vida y un seguro de salud?",
    "Comment fonctionne l'assurance automobile en cas d'accident?",
    "Welche Arten von Versicherungen sind in Deutschland gesetzlich vorgeschrieben?",
    "保险如何保护财产损失？",
    "Каковы основные виды страхования в России?",
    "ما هو التأمين على الحياة وما هي فوائده؟",
    "自動車保険の種類とは何ですか？",
    "Como funciona o seguro de saúde em Portugal?",
    "बीमा क्या होता है और यह कितने प्रकार का होता है?"
]

In [16]:
tru_llm_standalone_recorder = TruBasicApp(llm_open_ai, app_id="Training_20Feb_helpful", feedbacks=helpful_feedbacks)

In [17]:
tru.reset_database()
with tru_llm_standalone_recorder as recording:
    for prompt_input in helpful_evals:
        tru_llm_standalone_recorder.app(prompt_input)

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
App Training_20Feb_helpful was not present in database. Adding it.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are n

### Output

In [19]:
tru.get_records_and_feedback(app_ids=[])[0][['input','output','Language Match','Input Sentiment','Output Sentiment','Coherence']] 

,input,output,Language Match,Input Sentiment,Output Sentiment,Coherence
0,"""What types of insurance are commonly used to ...","""Some types of insurance commonly used to prot...",0.962431,1.0,1.0,0.9
1,"""\u00bfCu\u00e1l es la diferencia entre un seg...","""La principal diferencia entre un seguro de vi...",0.994292,0.0,1.0,0.9
2,"""Comment fonctionne l'assurance automobile en ...","""Lorsqu'un accident de voiture se produit, l'a...",0.996474,0.8,1.0,0.9
3,"""Welche Arten von Versicherungen sind in Deuts...","""In Deutschland sind folgende Arten von Versic...",0.998893,0.0,1.0,0.9
4,"""\u4fdd\u9669\u5982\u4f55\u4fdd\u62a4\u8d22\u4...","""\u4fdd\u9669\u53ef\u4ee5\u901a\u8fc7\u4ee5\u4...",0.969234,1.0,1.0,0.8
5,"""\u041a\u0430\u043a\u043e\u0432\u044b \u043e\u...","""\u0412 \u0420\u043e\u0441\u0441\u0438\u0438 \...",NaN,0.8,1.0,0.8
6,"""\u0645\u0627 \u0647\u0648 \u0627\u0644\u062a\...","""\u0627\u0644\u062a\u0623\u0645\u064a\u0646 \u...",NaN,0.8,1.0,0.8
7,"""\u81ea\u52d5\u8eca\u4fdd\u967a\u306e\u7a2e\u9...","""\u81ea\u52d5\u8eca\u4fdd\u967a\u306b\u306f\u3...",NaN,0.8,1.0,0.9
8,"""Como funciona o seguro de sa\u00fade em Portu...","""O seguro de sa\u00fade em Portugal funciona d...",0.999182,0.8,0.8,0.8
9,"""\u092c\u0940\u092e\u093e \u0915\u094d\u092f\u...","""\u092c\u0940\u092e\u093e \u090f\u0915 \u0935\...",0.999571,0.0,1.0,0.8


In [20]:
tru.get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,...,Input Sentiment,Output Sentiment,Coherence,Language Match_calls,Input Sentiment_calls,Output Sentiment_calls,Coherence_calls,latency,total_tokens,total_cost
0,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_c6473d67fdea0288b66d5680cd8cf3bd,"""What types of insurance are commonly used to ...","""Some types of insurance commonly used to prot...",-,"{""record_id"": ""record_hash_c6473d67fdea0288b66...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-20T13:29:20.636486"", ""...",...,1.0,1.0,0.9,[{'args': {'text1': 'What types of insurance a...,[{'args': {'text': 'What types of insurance ar...,[{'args': {'text': 'Some types of insurance co...,[{'args': {'text': 'Some types of insurance co...,3,0,0.0
1,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_3798ab452deef06f2d15bf701f6ead68,"""\u00bfCu\u00e1l es la diferencia entre un seg...","""La principal diferencia entre un seguro de vi...",-,"{""record_id"": ""record_hash_3798ab452deef06f2d1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-20T13:29:24.315383"", ""...",...,0.0,1.0,0.9,[{'args': {'text1': '¿Cuál es la diferencia en...,[{'args': {'text': '¿Cuál es la diferencia ent...,[{'args': {'text': 'La principal diferencia en...,[{'args': {'text': 'La principal diferencia en...,3,0,0.0
2,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_2b6d5ed95d04a51d512c46ddd52e920a,"""Comment fonctionne l'assurance automobile en ...","""Lorsqu'un accident de voiture se produit, l'a...",-,"{""record_id"": ""record_hash_2b6d5ed95d04a51d512...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-20T13:29:28.162480"", ""...",...,0.8,1.0,0.9,[{'args': {'text1': 'Comment fonctionne l'assu...,[{'args': {'text': 'Comment fonctionne l'assur...,[{'args': {'text': 'Lorsqu'un accident de voit...,[{'args': {'text': 'Lorsqu'un accident de voit...,4,0,0.0
3,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_ecede31a0d27ce1fb5083b9cb0a56b0d,"""Welche Arten von Versicherungen sind in Deuts...","""In Deutschland sind folgende Arten von Versic...",-,"{""record_id"": ""record_hash_ecede31a0d27ce1fb50...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-20T13:29:33.095069"", ""...",...,0.0,1.0,0.9,[{'args': {'text1': 'Welche Arten von Versiche...,[{'args': {'text': 'Welche Arten von Versicher...,[{'args': {'text': 'In Deutschland sind folgen...,[{'args': {'text': 'In Deutschland sind folgen...,4,0,0.0
4,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_cfb1360b471f487ea14375ad35cc4ce8,"""\u4fdd\u9669\u5982\u4f55\u4fdd\u62a4\u8d22\u4...","""\u4fdd\u9669\u53ef\u4ee5\u901a\u8fc7\u4ee5\u4...",-,"{""record_id"": ""record_hash_cfb1360b471f487ea14...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-20T13:29:37.476825"", ""...",...,1.0,1.0,0.8,"[{'args': {'text1': '保险如何保护财产损失？', 'text2': '保...","[{'args': {'text': '保险如何保护财产损失？'}, 'ret': 1.0,...",[{'args': {'text': '保险可以通过以下几种方式来保护财产损失： 1. 财...,[{'args': {'text': '保险可以通过以下几种方式来保护财产损失： 1. 财...,5,0,0.0
5,Training_20Feb_helpful,"{""tru_class_info"": {""name"": ""TruBasicApp"", ""mo...",TruWrapperApp(trulens_eval.tru_basic_app),record_hash_2346454e60dbf8a098c92c802eae5f8c,"""\u041a\u0430\u043a\u043e\u0432\u044b \u043e\u...","""\u0412 \u0420\u043e\u0441\u0441\u0438\u0438 \...",-,"{""record_id"": ""record_hash_2346454e60dbf8a098c...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_

### Understanding Context

In [21]:
tru.get_records_and_feedback(app_ids=[])[0]['Coherence_calls'].iloc[9]

[{'args': {'text': 'बीमा एक वित्तीय सुरक्षा की सुविधा है जिसमें एक व्यक्ति या संगठन एक निश्चित धनराशि को एक बीमा कंपनी को भुगतान करके सुरक्षित करवाता है। यह एक प्रकार की वित्तीय सुरक्षा होती है जो आपको आपके नुकसान के मामले में आराम देती है।\n\nबीमा कई प्रकार का होता है, जिनमें से कुछ प्रमुख प्रकार निम्नलिखित हैं:\n\n1. जीवन बीमा: यह बीमा एक व्यक्ति की मृत्यु के बाद उसके परिवार को आर्थिक सुरक्षा प्रदान करता है। इसमें व्यक्ति एक निश्चित धनराशि का भुगतान करता है और उसके मृत्यु के मामले में उसके परिवार को एक निश्चित राशि मिलती है।\n\n2. स्वास्थ्य बीमा: यह बीमा व्यक्ति को चिकित्सा खर्चों के लिए सुरक्षा प्रदान करता है। इसमें व्यक्ति एक निश्चित राशि का भुगतान करता है और उसे चिकित्सा खर्चों के लिए आराम दिया जाता है।\n\n3. वाहन बीमा: यह बीमा वाहनों के नुकसान के मामले में सुरक्षा प्रदान करता है। इसमें व्यक्ति एक निश्चित राशि का भुगतान करता है और उसे वाहन के नुकसान के मामले में आराम दिया जाता है।\n\n4. गृह बीमा: यह बीमा घर और उसकी संपत्ति के नुकसान के मामले में सुरक्षा प्रदान करता है। इसमें व्यक्

## Evaluation in RAG

In [11]:
# Imports main tools:
from trulens_eval import TruChain, Feedback, Tru
tru = Tru()
tru.reset_database()

# Imports from langchain to build app
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

In [12]:
from langchain_openai import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    azure_deployment='text-embedding-ada-002'
)

### Vector DB

In [25]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = TextLoader("input/Harry Potter 3 Prisoner of Azkaban.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)


In [46]:
db.save_local("input/vectordb/Harry_Potter_3")

Or Fetch From Local

In [13]:
vectorstore = FAISS.load_local('input/vectordb/Harry_Potter_3', embeddings)

### Langchain retriver

In [14]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = AzureChatOpenAI(model_name='gpt-35-turbo', temperature=0)
# llm = AzureChatOpenAI(model_name='ecv-gpt-4-32k', temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
rag_chain.invoke("Does James Potter created the patronous charm beside the lake?")

'Harry Potter believes that his father, James Potter, conjured the Patronus charm beside the lake.'

### Feedback Function

In [16]:
from trulens_eval.feedback.provider import AzureOpenAI
from trulens_eval.feedback import Groundedness

import numpy as np
# Initialize AzureOpenAI-based feedback function collection class:
azopenai = AzureOpenAI(
    deployment_name="gpt-35-turbo")

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(azopenai.relevance, name = "Answer Relevance").on_input_output()

from trulens_eval.app import App
context = App.select_context(rag_chain)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(azopenai.qs_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

# groundedness of output on the context
# Read the prompts here trulens_eval/trulens_eval/feedback/prompts.py
groundedness = Groundedness(groundedness_provider=azopenai)
f_groundedness = (
    Feedback(groundedness.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(groundedness.grounded_statements_aggregator)
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.app.first.steps.context.first.get_relevant_documents.rets .
✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps.context.first.get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [17]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_qa_relevance, f_context_relevance, f_groundedness])

In [22]:
with tru_recorder as recording:
    llm_response = rag_chain.invoke("Does James Potter created the patronous charm beside the lake?")

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.


In [23]:
rec = recording.get()

### Evaluation

In [24]:
for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

Answer Relevance 0.8
qs_relevance 0.7
groundedness_measure_with_cot_reasons 1.0


In [25]:
records, feedback = tru.get_records_and_feedback(app_ids=["Chain1_ChatApplication"])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,qs_relevance,groundedness_measure_with_cot_reasons,Answer Relevance_calls,qs_relevance_calls,groundedness_measure_with_cot_reasons_calls,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_df9bb3ba8ebbe4d5fe4e74dadaab4fea,"""who killed dumbledore?""","""I don't know.""",-,"{""record_id"": ""record_hash_df9bb3ba8ebbe4d5fe4...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-25T13:56:06.066288"", ""...",2024-02-25T13:56:06.682137,1.0,0.2,0.0,"[{'args': {'prompt': 'who killed dumbledore?',...",[{'args': {'question': 'who killed dumbledore?...,"[{'args': {'source': [[{'page_content': '""Siri...",0,0,0.0
1,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_70496b2e2f3c728fad14347fea784e00,"""Does James Potter created the patronous charm...","""Harry Potter believes that his father, James ...",-,"{""record_id"": ""record_hash_70496b2e2f3c728fad1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-02-25T13:56:46.516640"", ""...",2024-02-25T13:56:47.310702,0.8,0.7,1.0,[{'args': {'prompt': 'Does James Potter create...,[{'args': {'question': 'Does James Potter crea...,[{'args': {'source': [[{'page_content': 'She t...,0,0,0.0
